In [1]:
# organize the imports
import opentrons.execute
from opentrons import types 
import time
import numpy as np


In [2]:
import opentrons
opentrons.__file__

'/usr/lib/python3.7/site-packages/opentrons/__init__.pyc'

In [8]:
#!pip3 install zeroconf
#!pip3 install requests

# Connect to the microscope 
from microscope_client import MicroscopeClient 
microscope = MicroscopeClient(host="192.168.178.33", port=5000)


In [4]:
# Connect to the opentrons robot
protocol = opentrons.execute.get_protocol_api('2.9')
protocol.home()

# setup the pipets
pipette = protocol.load_instrument('p300_single_gen2', 'right')
pipette_8 = protocol.load_instrument('p300_multi', 'left') # causes errors -> don't use two pipettes!!


Failed to initialize character device, will not be able to control gpios (lights, button, smoothiekill, smoothie reset). Only one connection can be made to the gpios at a time. If you need to control gpios, first stop the robot server with systemctl stop opentrons-robot-server. Until you restart the server with systemctl start opentrons-robot-server, you will be unable to control the robot using the Opentrons app.
Out of bounds move: X=(418.00000000000006 motor controller, 419.5412429817696 deck) too high for limit 418.0
Out of bounds move: X=(418.00000000000006 motor controller, 419.5412429817696 deck) too high for limit 418.0


In [9]:
# setup the labware in the trays
#plate_microscope = protocol.load_labware('corning_96_wellplate_360ul_flat', location='5')
plate_reagents = protocol.load_labware('corning_24_wellplate_3.4ml_flat', location='9')

# Move light to the sample 
tiprack = protocol.load_labware('opentrons_96_filtertiprack_200ul',location='8')

# sample plate on microscope
# offset of the well plate in mm
offset_well_x = 100
offset_well_y = 10
offset_well_z = 10
plate_sample = protocol.load_labware('corning_96_wellplate_360ul_flat', location='3')

ValueError: Deck location 7 already  has an item: <opentrons.protocols.implementations.labware.LabwareImplementation object at 0x6eeaa210>

In [72]:
from opentrons.types import Location, Point

# set speed
protocol.max_speeds['X'] = 200 # default = 400
protocol.max_speeds['Y'] = 200 # default = 300


# ######
# 0 4 8
# 1 5 9
# 2 6 10 
# 3 7 11

# wells with reagents
i_phal = 0
i_sir = 1
i_pbs = 2
i_trash = 3

# general aspiration volume
V_aspirate = 200

# move it down since opentrons expects the reagents to be furthter away from the ground
offset_z_reagents = -20

# define locations 
minimum_z_height = 150 # minimum height for arcs driven by the robot
position_sample_light = (252, 100, 210)

# compute the positions
position_idle =  (200,200,140)#
position_phal = list(plate_reagents.well(i_phal).center()._point)
position_sir = list(plate_reagents.well(i_sir).center()._point)
position_pbs = list(plate_reagents.well(i_pbs).center()._point)
position_trash = list(plate_reagents.well(i_trash).center()._point)


def move_to_coord(pipette, position_xyz, offset = (0,0,0), minimum_z_height=190):
    if pipette._last_tip_picked_up_from is None:
        print("Please add a pipette first!")
        
        
    position_final = (position_xyz[0]+offset[0],
                        position_xyz[1]+offset[1],
                        position_xyz[2]+offset[2])
    print("Moving to: "+str(position_final))                       
    pipette.move_to(Location(Point(*position_final), None),minimum_z_height=minimum_z_height)


def pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, z_offset=-3):
    try:
        my_fresh_tip = tiprack.wells()[i_pipette]
        adjusted_location = my_fresh_tip.center().move(types.Point(x=0, y=0, z=z_offset))
        pipette.pick_up_tip(adjusted_location)
    except Exception as e:
        print(e)
        i_pipette += 1
        print("no pipette attached")
        pipette.drop_tip()
        pipette.pick_up_tip(my_fresh_tip)

    return i_pipette


def takesnapshot(pipette, position_sample_light, offset=(2,-2,0), I_led=1, I_laser=0):
    microscope.set_laser_led(I_laser,I_led)
    move_to_coord(pipette, position_sample_light, offset=offset, minimum_z_height=minimum_z_height)
    microscope.capture_image_to_disk()
    microscope.set_laser_led(0,0)
    
def wellscan(microscope, offset_x, offset_y, i_experiment,
        Nx=3, Ny=3, well_to_well_steps = 9000,
        autofocus_dz=2000, autofocus_Nz=11,
            name_experiment="wellscan_",
            focus_pos_list=None, is_autofocus=True, 
            I_laser=0, I_led=1):
    """
    Save a set of images in a wellscan

    Args:
        microscope: Microscope object
        offset_x (int): Number of images to take
        offset_y (int/float): Time, in seconds, between sequential captures
    """
    folder = "wellscan_"

    # cast variables
    Nx = int(Nx)
    Ny = int(Ny)
    t_period = int(t_period)
    well_to_well_steps = int(well_to_well_steps)
    
    # get current z-position for the stage
    offset_z = microscope.stage.position[-1]

    # move to the home positoin
    print("Start moving to the position")
    
    # go to the first location
    microscope.move((offset_x,offset_y,offset_z), absolute=True)

    # in case we have focus positions available, skip it
    print("Start scan")
    #%%
    if focus_pos_list is None:
        focus_pos_list = []
        
    folder = "WELLSCAN_{}".format(base_file_name)

    # preset variables
    i_image = 0
    i_well = 0
    last_offset_z_row = offset_z
    for wellpos_y in range(Nx):
        for wellpos_x in range(Nx):

            if last_offset_z_row == 0:
                offset_z = last_offset_z_row

            print("Move microscope")
            current_x, current_y = offset_x+well_to_well_steps*wellpos_x,offset_y+well_to_well_steps*wellpos_y

            if is_autofocus:
                microscope.move((current_x, current_y, offset_z), absolute=True)
                
                # turn on laser/led and perform autofocus
                microscope.set_laser_led(I_laser,I_led)
                offset_z = autofocus_coarse(dz=autofocus_dz, nz=autofocus_Nz, is_uselight=True)
                microscope.set_laser_led(0,0)                
                focus_pos_list.append(offset_z)

                if last_offset_z_row == 0:
                    last_offset_z_row = offset_z
            else:
                offset_z = focus_pos_list[i_well]
                microscope.move((current_x, current_y, offset_z), absolute=True)

            print("offset_z:"+str(offset_z))
            
            # debounce
            time.sleep(.5)
            
            # turn on light and take an image
            if I_laser > 0:
                params = {
                    "use_video_port": True,
                    "bayer": False,
                    "temporary" :False,
                    "tags": "xy_scan_"+str(wellpos_x)+"_"+str(wellpos_y),
                    "filename": name_experiment+"FLOU_"+str(i_experiment)+"_"+
                                str(i_well)+"_"+str(i_image)+
                                "_"+str(wellpos_x)+"_"+str(wellpos_y), 
                    "folder": folder
                }
                microscope.set_laser_led(I_laser,0)
                microscope.capture_image_to_disk(params)
            if I_led > 0:
                params = {
                    "use_video_port": True,
                    "bayer": False,
                    "temporary" :False,
                    "tags": "xy_scan_"+str(wellpos_x)+"_"+str(wellpos_y),
                    "filename": name_experiment+"BF_"+str(i_experiment)+"_"+
                                str(i_well)+"_"+str(i_image)+
                                "_"+str(wellpos_x)+"_"+str(wellpos_y), 
                    "folder": folder
                }
                microscope.set_laser_led(0,1)
                microscope.capture_image_to_disk(params)
            microscope.set_laser_led(0,0)            
            print(filename)
            i_image += 1
            i_well += 1

    i_experiment += 1
    return focus_pos_list

def autofocus_coarse(dz=1000, nz=11, is_uselight=False):
    if is_uselight:
        microscope.set_laser_led(0,1)
        time.sleep(.2)
    payload = {'dz': list(np.linspace(-dz,dz,nz))}
    print("Focusing: "+str(payload))
    microscope.extensions["org.openflexure.autofocus"]["autofocus"].post_json(payload)
    if is_uselight:
        microscope.set_laser_led(0,1)
    return microscope.position['z']


In [51]:
#protocol.home()
i_pipette = 0 
home_position_microscope=(0,0,0)


Please add a pipette first!
Moving to: (252, 163, 200)


In [74]:
# autofocus_coarse(dz=1000, nz=5, is_uselight=True)

Focusing: {'dz': [-1000.0, -500.0, 0.0, 500.0, 1000.0]}


1889

# Do a time-lapse for an hour

In [ ]:
# Microscope coordinates
offset_x = 8000
offset_y = 4000
i_experiment = 0
Nx=8
Ny=3
well_to_well_steps = 9000
autofocus_dz = 2000
autofocus_Nz=11

mytime = time.time()
i_timelapse_before = 1
t_duration = 60 # minutes
focus_pos_list = None # start with a fresh list
while ((time.time()-mytime)<(t_duration*60)):
    time_now = time.time()
    
    # perform a testing well scan by moving robot to light position and do a whole plate scan
    move_to_coord(pipette_8, position_sample_light, offset=(0,0,0), minimum_z_height=minimum_z_height)
    focus_pos_list = wellscan(microscope, offset_x, offset_y, i_experiment,
            Nx, Ny, well_to_well_steps,
            autofocus_dz, autofocus_Nz,
                name_experiment="wellscan_",
                focus_pos_list=focus_pos_list, is_autofocus=True, 
                I_laser=0, I_led=1)

In [58]:
microscope.set_laser_led(0,0)

# Add AF647Phal

In [34]:
V_aspirate = 100

# move the wellplate to the zero position with the microscope
microscope.move(home_position_microscope, absolute=True)

# 1. Pickup new pipette
i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, -8)

V_aspirates_phal = (10,20,50,75,100,150,200,250)
for i_pos in range(len(plate_sample.rows())):
    
    # 2. Pickup Reagent depending on current iteration
    move_to_coord(pipette, position_phal, offset=(0,0,offset_z_reagents), minimum_z_height=minimum_z_height)
    pipette.aspirate(V_aspirates_phal[i_pos])
    
    # 3. Move to Sample location and mix it with the sample
    position_sample_i = np.array(plate_sample.rows()[i_phal][i_pos].center()._point)+np.array((offset_well_x,offset_well_y,offset_well_z))
    move_to_coord(pipette, position_sample_i, offset=(0,0,offset_z_reagents), minimum_z_height=minimum_z_height)
    pipette.dispense()
    pipette.mix(3)


Cannot pick up tip with a tip attached
no pipette attached


Moving to: (17.48, 214.07, -8.43)


Moving to: (252, 133, 131)


Moving to: (17.48, 214.07, -8.43)


# Add SiR

In [35]:
V_aspirate = 100

# move the wellplate to the zero position with the microscope
microscope.move(home_position_microscope, absolute=True)

# 1. Pickup new pipette
i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, -8)

V_aspirates_sir = (10,20,50,75,100,150,200,250)
for i_pos in range(len(plate_sample.rows())):
    
    # 2. Pickup Reagent depending on current iteration
    move_to_coord(pipette, position_sir, offset=(0,0,offset_z_reagents), minimum_z_height=minimum_z_height)
    pipette.aspirate(V_aspirates_sir[i_pos])
    
    # 3. Move to Sample location and mix it with the sample
    position_sample_i = np.array(plate_sample.rows()[1i_sir][i_pos].center()._point)+np.array((offset_well_x,offset_well_y,offset_well_z))
    move_to_coord(pipette, position_sample_i, offset=(0,0,offset_z_reagents), minimum_z_height=minimum_z_height)
    pipette.dispense()
    pipette.mix(3)


Moving to: (252, 133, 131)


Moving to: (36.78, 252.67000000000002, 11.57)


# Do timelapse for an hour

In [38]:
# Microscope coordinates
mytime = time.time()
i_timelapse_before = 1
t_duration = 60 # minutes
focus_pos_list = None # start with a fresh list
while ((time.time()-mytime)<(t_duration*60)):
    time_now = time.time()
    
    # perform a testing well scan by moving robot to light position and do a whole plate scan
    move_to_coord(pipette_8, position_sample_light, offset=(0,0,0), minimum_z_height=minimum_z_height)
    focus_pos_list = wellscan(microscope, offset_x, offset_y, i_experiment,
            Nx, Ny, well_to_well_steps,
            autofocus_dz, autofocus_Nz,
                name_experiment="wellscan_",
                focus_pos_list=focus_pos_list, is_autofocus=True, 
                I_laser=0, I_led=1)

Please add a pipette first!
Moving to: (252, 166, 200)
(252, 166, 200)


# Wash sample 3x 

In [100]:
pipette.home()
i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack)

# add primary to sample
V_aspirate = 200

# move the wellplate to the zero position with the microscope
microscope.move(home_position_microscope, absolute=True)

for i in range(3):
    # Do that 3 times
    for i_pos in range(len(plate_sample.rows()[0:3])):
        
        # 2. Move to Sample location and aspirate liquid 
        position_sample_i = np.array(plate_sample.wells()[0].center()._point)+np.array((offset_well_x,offset_well_y,offset_well_z))
        move_to_coord(pipette, position_sample_i, offset=(0,0,0), minimum_z_height=minimum_z_height)
        pipette.aspirate(V_aspirate)

        #
        # 3. Trash the liquid
        move_to_coord(pipette, position_trash, offset=(0,0,offset_z_reagents), minimum_z_height=minimum_z_height)
        pipette.dispense()
        pipette.blow_out()
        
        # 4. grab PBS and aspirate
        move_to_coord(pipette, position_pbs, offset=(0,0,offset_z_reagents), minimum_z_height=minimum_z_height)
        pipette.aspirate(V_aspirate)

        # 5. go to microscope and dispense
        position_sample_i = np.array(plate_sample.wells()[0].center()._point)+np.array((offset_well_x,offset_well_y,offset_well_z))
        move_to_coord(pipette, position_sample_i, offset=(0,0,0), minimum_z_height=minimum_z_height)
        pipette.dispense(V_aspirate)        
        pipette.blow_out()        
    

Moving to: (17.48, 252.67000000000002, -3.4299999999999997)


Moving to: (252, 133, 131)


TypeError: move_to_coord() missing 1 required positional argument: 'position_xyz'

# Do a timelapse with fluorescence

In [ ]:
# Microscope coordinates
mytime = time.time()
i_timelapse_before = 1
t_duration = 60 # minutes
focus_pos_list = None # start with a fresh list
while ((time.time()-mytime)<(t_duration*60)):
    time_now = time.time()
    
    # perform a testing well scan by moving robot to light position and do a whole plate scan
    move_to_coord(pipette_8, position_sample_light, offset=(0,0,0), minimum_z_height=minimum_z_height)
    focus_pos_list = wellscan(microscope, offset_x, offset_y, i_experiment,
            Nx, Ny, well_to_well_steps,
            autofocus_dz, autofocus_Nz,
                name_experiment="wellscan_",
                focus_pos_list=focus_pos_list, is_autofocus=True, 
                I_laser=255, I_led=1)